In [1]:
import pandas as pd
from os import listdir

In [2]:
# Coletando nomes dos arquivo CSV presentes na pasta "path_to_dir"
path_to_dir = "data/BDTD"
filenames = listdir(path_to_dir)
csv_files = [ path_to_dir + "/" + filename for filename in filenames if filename.endswith( ".csv" ) ]

In [3]:
# Concatenando todos os arquivos
data = pd.DataFrame()
for csv in csv_files:
    data = pd.concat([data, pd.read_csv(csv)])
data = data.drop_duplicates(subset=['Título'], keep='first').reset_index(drop=True)

In [4]:
# Quebrando assuntos em lista
data["Assuntos em português"] = data["Assuntos em português"].apply(lambda x: x.split("||"))
data["Assuntos em inglês"] = data["Assuntos em inglês"].apply(lambda x: x.split("||"))

In [5]:
# Limpando o nome dos autores
autors = data["Autor(a)"].apply(lambda x: x.split("||"))

new_autors = []
for n in range(len(autors)):
    new_autors.append(autors[n][0])
    if len(autors[n]) > 1:
        if autors[n][1].startswith("http://lattes.cnpq.br/"):
            data.loc[n, "ID Lattes do(a) autor(a)"] = autors[n][1]

data["Autor(a)"] = new_autors

In [6]:
# Limpando os nomes dos orientadores

orientadores = data["Orientadores"].apply(lambda x: x.split("||"))

for n in range(len(orientadores)):
    if "Não informado pela instituição" in orientadores[n]:
        orientadores[n] = list(filter(lambda a: a != "Não informado pela instituição", orientadores[n]))

data["Orientadores"] = orientadores

In [7]:
# Limpando os nomes dos membros da banca

banca = data["Membros da banca"].apply(lambda x: x.split("||"))

for n in range(len(banca)):
    if "Não informado pela instituição" in banca[n]:
        banca[n] = list(filter(lambda a: a != "Não informado pela instituição", banca[n]))

data["Membros da banca"] = banca

In [8]:
# Salvando como csv
path_to_dir_output = "data"
data.to_csv(path_to_dir_output + "/BDTD_Consolidado.csv", index=False)